In [21]:
# Importy
from ucimlrepo import fetch_ucirepo
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, f1_score
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
import csv
import re

In [22]:
# Pobranie danych
heart_disease = fetch_ucirepo(id=45)

# Porzucenie linii z pustymi etykietami oraz odpowiadajacych im wartosci
feature_matrix = heart_disease.data.features.dropna()
labels = heart_disease.data.targets.loc[feature_matrix.index]

# Przetworzenie zbioru wartości przewidywanych do wartości binarnych
y_binary = labels.copy()
y_binary['num'] = y_binary['num'].apply(lambda x: 1 if x != 0 else 0)                      

# Utworznnie zbioru dummy etykiet
x_dummy = pd.get_dummies(feature_matrix, columns=['cp', 'restecg', 'slope','ca','thal'])   

# upewnienie się co do typów wykorzystywanych danych
x_dummy = x_dummy.fillna(0).astype(float)
y_binary = y_binary.astype(float)

# Podział danych na zbiór uczący i testowy
x_train, x_test, y_train, y_test = train_test_split(x_dummy, y_binary, test_size=0.2, random_state=268555)

scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

# Konwersja danych do tablic NumPy przed utworzeniem tensorów PyTorch
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.float32).unsqueeze(1)  # wektor kolumnowy
x_test_tensor = torch.tensor(x_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test.values, dtype=torch.float32).unsqueeze(1)    # wektor kolumnowy

In [23]:
# Stworzona klasa Dataset
class HeartDiseaseDataset(Dataset):
    def __init__(self, features, labels):
        self.features = features
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

# Instancja klasy Dataset
train_dataset = HeartDiseaseDataset(x_train_tensor, y_train_tensor)
test_dataset = HeartDiseaseDataset(x_test_tensor, y_test_tensor)

# Stworzony model sieci neuronowej
class HeartDiseaseModel(nn.Module):
    def __init__(self, input_dim, layer_one_dim = 0, layer_two_dim = 0, layer_three_dim = 0, layer_four_dim = 0):
        super(HeartDiseaseModel, self).__init__()
        self.layer_one_dim = layer_one_dim
        self.layer_two_dim = layer_two_dim
        self.layer_three_dim = layer_three_dim
        self.layer_four_dim = layer_four_dim
        self.relu = nn.ReLU()
        
        if (self.layer_one_dim == 0):    self.fc1 = nn.Linear(input_dim, 1)
        else:
            self.fc1 = nn.Linear(input_dim, layer_one_dim)
            if (self.layer_two_dim == 0):    self.fc2 = nn.Linear(layer_one_dim, 1)
            else:
                self.fc2 = nn.Linear(layer_one_dim, layer_two_dim)

                if (self.layer_three_dim == 0):  self.fc3 = nn.Linear(layer_two_dim, 1)
                else:
                    self.fc3 = nn.Linear(layer_two_dim, layer_three_dim)

                    if (self.layer_four_dim == 0):    self.fc4 = nn.Linear(layer_three_dim, 1)
                    else:
                        self.fc4 = nn.Linear(layer_three_dim, layer_four_dim)
                        self.fc5 = nn.Linear(layer_four_dim, 1)
                        

        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.fc1(x)
        if (self.layer_one_dim == 0):    x = self.sigmoid(x)
        else:
            x = self.relu(x)
            x = self.fc2(x)
            if (self.layer_two_dim == 0):    x = self.sigmoid(x)
            else:
                x = self.relu(x)
                x = self.fc3(x)
                if (self.layer_three_dim == 0):  x = self.sigmoid(x)
                else:
                    x = self.relu(x)
                    x = self.fc4(x)
                    if (self.layer_four_dim != 0):
                        x = self.relu(x)
                        x = self.fc5(x)
                    x = self.sigmoid(x)

        return x

# Funkcja kosztu
criterion = nn.BCELoss()

# Trenowanie modelu
def train_model(model, optimizer, num_epochs=100):
    for epoch in range(num_epochs):
        model.train()
        total_cost = 0.0
        for inputs, labels in train_loader:
            labels = labels.squeeze(1)  # Usunięcie dodatkowego wymiaru
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            total_cost += loss.item()
        
        if(epoch % 10 == 9):
            print(f"Epoch [{epoch+1}/{num_epochs}], Cost: {total_cost / len(train_loader):.4f}")

# Ewaluacja modelu
def evaluate_model(model, loader, threshold=0.5):
    model.eval()
    all_preds = []
    all_labels = []
    with torch.no_grad():
        for inputs, labels in loader:
            outputs = model(inputs)
            preds = (outputs > threshold).float()
            all_preds.extend(preds.numpy().flatten())
            all_labels.extend(labels.numpy().flatten())

    accuracy = accuracy_score(all_labels, all_preds)
    f1 = f1_score(all_labels, all_preds, zero_division=1)
    precision = precision_score(all_labels, all_preds, zero_division=1)
    return accuracy, f1, precision

<h4> Ze względów praktycznych wartości parametrów różnią się względem poprzedniej listy, nie mniej jednak sprawdzono wpływ poszczególnych czynników na ogólne działanie

In [24]:
# batch_size, learning_rate, layer_one_dim, layer_two_dim, layer_three_dim, layer_four_dim
parameters = [
    # zmiany batch_size
    [64, 0.001, 8, 4, 0, 0],
    [32, 0.001, 8, 4, 0, 0],
    [16, 0.001, 8, 4, 0, 0],

    # zmiany learning_rate
    [64, 0.01, 8, 4, 0, 0],
    [64, 0.0001, 8, 4, 0, 0],

    # zmiany w ilości perceptonów
    [64, 0.001, 8, 4, 0, 0],
    [64, 0.001, 12, 6, 2, 0],
    [64, 0.001, 6, 0, 0, 0],
    [64, 0.001, 12, 6, 0, 0],
    [64, 0.001, 12, 8, 4, 2],
    [64, 0.001, 4, 2, 0, 0]
]

In [25]:
# Trenowanie i ewaluacja z użyciem różnych optymalizatorów
results = {}

for parameter_set in  parameters:
    # instancja DataLoader
    train_loader = DataLoader(train_dataset, batch_size=parameter_set[0], shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=parameter_set[0], shuffle=True)

    # instancja modelu
    model = HeartDiseaseModel(x_train_tensor.shape[1], parameter_set[2], parameter_set[3], parameter_set[4], parameter_set[5])

    # optymalizatory
    optimizers = {
        "SGD": optim.SGD(model.parameters(), lr=parameter_set[1], momentum=0.9),
        "Adam": optim.Adam(model.parameters(), lr=parameter_set[1]),
        "RMSprop": optim.RMSprop(model.parameters(), lr=parameter_set[1], momentum=0.9)
    }

    for name, optimizer in optimizers.items():
        print(f"\nTraining with {name} optimizer.\nBatch: {parameter_set[0]}\nLearning: {parameter_set[1]}\nNetwork: [{parameter_set[2]}, {parameter_set[3]}, {parameter_set[4]}, {parameter_set[5]}]")
        train_model(model, optimizer)
        accuracy, f1, precision = evaluate_model(model, test_loader)
        results["Name:%sBatch:%sLearning:%sNetwork:%s" % (name, parameter_set[0], parameter_set[1], [parameter_set[2], parameter_set[3], parameter_set[4], parameter_set[5]])] = {
            "accuracy": accuracy,
            "f1_score": f1,
            "precision": precision
        }

results


Training with SGD optimizer.
Batch: 64
Learning: 0.001
Network: [8, 4, 0, 0]
Epoch [10/100], Cost: 0.7308
Epoch [20/100], Cost: 0.7034
Epoch [30/100], Cost: 0.6830
Epoch [40/100], Cost: 0.6691
Epoch [50/100], Cost: 0.6482
Epoch [60/100], Cost: 0.6315
Epoch [70/100], Cost: 0.6139
Epoch [80/100], Cost: 0.5952
Epoch [90/100], Cost: 0.5754
Epoch [100/100], Cost: 0.5540

Training with Adam optimizer.
Batch: 64
Learning: 0.001
Network: [8, 4, 0, 0]
Epoch [10/100], Cost: 0.4899
Epoch [20/100], Cost: 0.4289
Epoch [30/100], Cost: 0.3824
Epoch [40/100], Cost: 0.3564
Epoch [50/100], Cost: 0.3242
Epoch [60/100], Cost: 0.3185
Epoch [70/100], Cost: 0.3029
Epoch [80/100], Cost: 0.2929
Epoch [90/100], Cost: 0.2851
Epoch [100/100], Cost: 0.2686

Training with RMSprop optimizer.
Batch: 64
Learning: 0.001
Network: [8, 4, 0, 0]
Epoch [10/100], Cost: 0.1663
Epoch [20/100], Cost: 0.1022
Epoch [30/100], Cost: 0.0601
Epoch [40/100], Cost: 0.0350
Epoch [50/100], Cost: 0.0212
Epoch [60/100], Cost: 0.0137
Epoch

{'Name:SGDBatch:64Learning:0.001Network:[8, 4, 0, 0]': {'accuracy': 0.8166666666666667,
  'f1_score': 0.7317073170731707,
  'precision': 0.9375},
 'Name:AdamBatch:64Learning:0.001Network:[8, 4, 0, 0]': {'accuracy': 0.8833333333333333,
  'f1_score': 0.8571428571428571,
  'precision': 0.875},
 'Name:RMSpropBatch:64Learning:0.001Network:[8, 4, 0, 0]': {'accuracy': 0.85,
  'f1_score': 0.8301886792452831,
  'precision': 0.7857142857142857},
 'Name:SGDBatch:32Learning:0.001Network:[8, 4, 0, 0]': {'accuracy': 0.8666666666666667,
  'f1_score': 0.84,
  'precision': 0.84},
 'Name:AdamBatch:32Learning:0.001Network:[8, 4, 0, 0]': {'accuracy': 0.8666666666666667,
  'f1_score': 0.84,
  'precision': 0.84},
 'Name:RMSpropBatch:32Learning:0.001Network:[8, 4, 0, 0]': {'accuracy': 0.9,
  'f1_score': 0.88,
  'precision': 0.88},
 'Name:SGDBatch:16Learning:0.001Network:[8, 4, 0, 0]': {'accuracy': 0.8666666666666667,
  'f1_score': 0.84,
  'precision': 0.84},
 'Name:AdamBatch:16Learning:0.001Network:[8, 4, 0,

In [26]:
def save_results_to_csv(results, filename='results.csv'):
    headers = ['name', 'batch_size', 'learning_rate', 'neural_network', 'accuracy', 'f1_score', 'precision']

    with open(filename, mode='w', newline='') as file:
        writer = csv.writer(file)
        writer.writerow(headers)
        for key, metrics in results.items():
            match = re.match(r"Name:(.+)Batch:(\d+)Learning:(\d*\.?\d+)Network:\[(.+)\]", key)
            if match:
                name, batch_size, learning_rate, neural_network = match.groups()
                row = [
                    name.strip(),
                    int(batch_size),
                    float(learning_rate),
                    neural_network.strip(),
                    metrics['accuracy'],
                    metrics['f1_score'],
                    metrics['precision']
                ]
                writer.writerow(row)

# Przykład użycia
save_results_to_csv(results)


<h3> Wnioski: </h3>
Zdecydowanie najlepszymi wynikami wyróżnia się optymalizator RMSprop, który potrafi osiągać wartości współczynników na poziomie 90%!. Nie zauważyłem znaczącej korelacji pomiędzy wartością parametrów uczenia i rozmiaru batcha a wynikami. Zarówno sieci z małą ilością warstw ukrytych i perceptonów, jak również ich odwrotności potrafiły poradzić sobie z zadaniem. Zaledwie 5 na 30 wyników osiągnęło średnią wartość parametrów na poziomie niższym niż 80% (wszystkie z nich korzystały z optymalizera SGD)